In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="viridis")

df = pd.read_csv("../data/raw/spain_energy.csv")

df.head()

,datetime,id,name,geoid,geoname,value
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500


In [3]:
df.rename(columns={
    'datetime': 'fecha_hora',
    'id': 'id_registro',
    'name': 'nombre_variable',
    'geoid': 'id_geografico',
    'geoname': 'nombre_region',
    'value': 'precio_energia'
}, inplace=True)

df.head()

,fecha_hora,id_registro,nombre_variable,id_geografico,nombre_region,precio_energia
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500


In [4]:
df.isnull().sum()

fecha_hora             0
id_registro            0
nombre_variable     5478
id_geografico      29256
nombre_region      29256
precio_energia         0
dtype: int64

In [5]:
df['nombre_variable'].unique()

array(['Precio mercado SPOT Diario ESP', 'Precio mercado SPOT Diario FRA',
       'Precio mercado SPOT Diario POR',
       'Energía asignada en Mercado SPOT Diario España',
       'Energía asignada en Mercado SPOT Diario Francia', nan,
       'Rentas de congestión mecanismos implícitos diario Francia exportación',
       'Rentas de congestión mecanismos implícitos diario Portugal exportación',
       'Rentas de congestión mecanismos implícitos diario Francia importación',
       'Rentas de congestión mecanismos implícitos diario Portugal importación',
       'Demanda real', 'Demanda programada PBF total',
       'Generación programada PBF total',
       'Generación programada PBF Eólica',
       'Generación programada PBF Ciclo combinado',
       'Generación programada PBF Carbón',
       'Generación programada PBF Nuclear',
       'Generación programada PBF Gas Natural Cogeneración',
       'Generación programada PBF UGH + no UGH',
       'Generación programada PBF Solar fotovoltaica'

In [6]:
df['id_geografico'].unique()

array([ 3.,  2.,  1., nan])

In [7]:
df['nombre_region'].unique()

array(['España', 'Francia', 'Portugal', nan], dtype=object)

In [8]:
def inferir_region(row):
    nombre = str(row['nombre_variable']).upper()
    if 'ESP' in nombre:
        return 'España'
    elif 'FRA' in nombre:
        return 'Francia'
    elif 'POR' in nombre:
        return 'Portugal'
    else:
        return row['nombre_region']

df['nombre_region'] = df.apply(inferir_region, axis=1)

In [9]:
mapa_ids = {'España': 3.0, 'Francia': 1.0, 'Portugal': 2.0}
df['id_geografico'] = df['nombre_region'].map(mapa_ids)


In [10]:
df = df.dropna(subset=['nombre_variable'])


In [12]:
df.isnull().sum()
df['nombre_region'].value_counts()
df['nombre_variable'].nunique()


20

In [14]:
pd.crosstab(df['nombre_region'], df['nombre_variable'])


nombre_variable,Energía asignada en Mercado SPOT Diario España,Energía asignada en Mercado SPOT Diario Francia,Precio mercado SPOT Diario ESP,Precio mercado SPOT Diario FRA,Precio mercado SPOT Diario POR,Rentas de congestión mecanismos implícitos diario Francia exportación,Rentas de congestión mecanismos implícitos diario Francia importación,Rentas de congestión mecanismos implícitos diario Portugal exportación,Rentas de congestión mecanismos implícitos diario Portugal importación
nombre_region,,,,,,,,,
España,1826,0,1826,0,0,0,0,0,0
Francia,0,1826,0,1826,0,712,1619,0,0
Portugal,0,0,0,0,1826,0,0,1566,1664


| País / Variable | Energía asignada                                  | Precio SPOT | Rentas de congestión                                                    |
| --------------- | ------------------------------------------------- | ----------- | ----------------------------------------------------------------------- |
| **España**      | 1.826 registros de energía y 1.826 de precio SPOT |             | Ninguna renta de congestión                                             |
| **Francia**     | 1.826 registros de energía y 1.826 de precio SPOT |             | Rentas de congestión Francia exportación (712) + importación (1.619)    |
| **Portugal**    | 1.826 registros de precio SPOT                    |             | Rentas de congestión Portugal exportación (1.566) + importación (1.664) |


In [16]:
df['fecha_hora'].min(), df['fecha_hora'].max()


('2014-01-01 23:00:00', '2018-12-31 23:00:00')

In [17]:
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
df.sort_values(by='fecha_hora', inplace=True)
df.set_index('fecha_hora', inplace=True)

In [ ]:
df.index.min(), df.index.max()
